### Task Move Operator:

The task move operator creates a neighbour by choosing a random job from an engineer’s schedule and assigning it to another engineer’s schedule while making sure the job is placed in the best position that minimizes the total time taken to complete the schedule. It has a complexity of O(n2).

The output of test is printed out at the end of the notebook

In [1]:
#Importing necessary libraries
import re
import time
import warnings
import numpy as np
import pandas as pd
import math
import requests
import json
import geocoder as gc
import sys
import os
from geopy import distance 
from geopy.geocoders import Nominatim

#Inputs
st = time.time()
sst = time.process_time()

class Engineer:
    def __init__(self,eid,location,skill,schedule,time):
        self.eid = eid
        self.location = location
        self.skill = skill
        self.schedule =schedule
        self.time = time
    
    def print(self):
        print("------------------------------------")
        print("E.ID =", self.eid)
        print("Schedule =", self.schedule)
        print("Total Time = {} minutes".format(self.time))
        
    def __repr__(self):
        output = '\n'
        output += '\n{eid:'
        output += self.eid
        output += '\nlocation:'
        output += self.location
        output += '\nskill: ['
        for i in self.skill:
            output += str(i)
            output += ','
        output += ']'
        output += '\nschedule: ['
        for i in self.schedule:
            output += str(i)
            output += ','
        output += ']}'
        return output
    
    def __str__(self):
        output = ''
        output += '\n{eid:'
        output += self.eid
        output += '\nlocation:'
        output += self.location
        output += '\nskill: ['
        for i in self.skill:
            output += str(i)
            output += ','
        output += ']'
        output += '\nschedule: ['
        for i in self.schedule:
            output += str(i)
            output += ','
        output += ']}'
        return output

class Jobs:
    
    def __init__(self,jid,location,time,skill):
        self.jid = jid
        self.location = location
        self.time = time
        self.skill = skill
        
    def __repr__(self):
        return self.jid


J1 = Jobs("1","DY11 5SW",120,2)
J2 = Jobs("2","WV4 6ED",160,2)
J3 = Jobs("3","B79 7PB",60,2)
J4 = Jobs("4","B90 8AT",120,1)
J5 = Jobs("5","B46 1AN",120,1)
J6 = Jobs("6","B98 9EY",120,1)
J7 = Jobs("7","NN1 5BD",240,3)
J8 = Jobs("8","B96 6BD",120,4)
J9 = Jobs("9","WV7 3BW",60,2)
J10 = Jobs("10","CV9 1LQ",90,2)
Job = Jobs("11","B16 9HN",90,2)

E1 = Engineer("1","B21 9RJ",[1,2],["3","10","5"],0)
E2 = Engineer("2","B68 0LH",[2,3],["2","9","1"],0)
E3 = Engineer("3","B11 1LZ",[1,4],["8","6","4"],0)
E4 = Engineer("4","B25 8RN",[1,3],["7"],0)

data = {
    "jobs":[J1,J2,J3,J4,J5,J6,J7,J8,J9,J10], "engg":[E1,E2,E3,E4]
}

#Function which returns job details from job id
def get_job(key,arr):
    for i in arr:
        if(i.jid == key):
            return i
    return None

#Function to get qualified engineers
def get_qualified(data,job):
    qualified = []
    for i in range(len(data["engg"])):
        if(job.skill in data["engg"][i].skill):
            qualified.append(i)
    return qualified

#OSRM TSP function:
def osrm_tsp(jobs,data): 
    lat = []
    long = []
    addresses = [data.location]
    for j in jobs:
        addresses.append(j.location)
    addresses.append(data.location)
    print("\nAddresses of the Jobs: ")
    print(addresses)
    locator = Nominatim(user_agent="myGeocoder")
    location = locator.geocode(data.location)
    lat.append(location.latitude)
    long.append(location.longitude)
    for i in jobs:
        locator = Nominatim(user_agent="myGeocoder")
        location = locator.geocode(i.location)
        lat.append(location.latitude)
        long.append(location.longitude)
    json_data = []
    lurl = ""
    for i in range(len(jobs)+1):
        lurl += f'{long[i]},{lat[i]};'
    url = 'http://127.0.0.1:5000/trip/v1/driving/'
    url += lurl[:-1]
    url += '?source=first'
    r = requests.get(url)
    json_data.append(r.json())
    for i in range(len(json_data[0]['waypoints'])):
        if(i==0):
            continue
        json_data[0]['waypoints'][i].update({"JID":jobs[i-1].jid})
    json_data[0]['waypoints'].sort(key = lambda x:x["waypoint_index"])
    schedule = []
    for i in json_data[0]['waypoints']:
        if('JID' in i):
            schedule.append(i['JID'])
    outputs = {
        "Duration": json_data[0]['trips'][0]['duration'],
        "Schedule": schedule
    }
    return outputs

#Function to calculate time of schedules generated by TSP function
def calculate_time_tsp(arr,data):
    job_time = 0
    for i in arr:
        job_time += i.time
    schedule = []
    print("\nChecking for schedule: ")
    for i in arr:
        schedule.append(i.jid)
    print(schedule)
    maps = osrm_tsp(arr,data)
    travel_time = maps["Duration"]
    if (travel_time == -1):
        print("Route not Possible")
        return -1
    travel_time = travel_time/60
    maps["Duration"] = job_time + travel_time
    return maps

#Function to calculate total time of each schedule
def icalculate_time(data):
    for i in data["engg"]:
        job_time = 0
        i.time = 0
        for j in i.schedule:
            job_time += get_job(j,data["jobs"]).time
        addresses = [i.location]
        for j in i.schedule:
            addresses.append(get_job(j,data["jobs"]).location)
        addresses.append(i.location)
        travel_time = osrm_route(addresses)
        i.time = travel_time/60 + job_time

#Task Move Operator        
def task_move_op_tsp(data,job,qualified):
    print("The Engineers qualified for Job N :")
    qe = [x+1 for x in qualified]
    print(qe)
    for i in qualified:
        print("\nTask Moving in Engineer: {}".format(data["engg"][i].eid))
        for j in range(len(data["engg"][i].schedule)):
            arr = []
            for k in range(len(data["engg"][i].schedule)):
                if(k==j):
                    continue
                arr.append(get_job(data["engg"][i].schedule[k],data["jobs"]))
            arr.append(job)
            maps = calculate_time_tsp(arr,data["engg"][i])
            time = maps["Duration"]
            print("Total Time Required : {} \n".format(time))
            if( time <= 480 and time > 0):
                print("Assigning Task Moved Job to another Engineer :\n")
                job_1 = get_job(data["engg"][i].schedule[j],data["jobs"])
                qualified_1 = get_qualified(data,job_1)
                print("The Engineers qualified for the moved Job :\n")
                qqe = [x+1 for x in qualified_1]
                print(qqe)
                print("\n")
                for l in qualified_1:
                    if(i!=l):
                        print("Attempting to assign job to Engineer {}:\n".format(data["engg"][l].eid))
                        arr_1 = []
                        for n in range(len(data["engg"][l].schedule)):
                            arr_1.append(get_job(data["engg"][l].schedule[n],data["jobs"]))
                        arr_1.append(job_1)
                        maps_1 = calculate_time_tsp(arr_1,data["engg"][l])
                        time_1 = maps_1["Duration"]
                        print("Total Time Required : {} \n".format(time_1))
                        if( time_1 <= 480 and time_1 > 0):
                            data["engg"][i].schedule = maps["Schedule"]
                            data["engg"][i].time = maps["Duration"]
                            data["engg"][l].schedule = maps_1["Schedule"]
                            data["engg"][l].time = maps_1["Duration"]
                            data["jobs"].append(job)
                            print("Job N Assigned Successfully\n")
                            return data 
                print("Not Possible \n")
        print("Task Move Failed \n")
    return -1

#OSRM routing function
def osrm_route(jobs): 
    lat = []
    long = []
    print("\nAddresses of the Jobs: ")
    print(jobs)
    for i in range(len(jobs)):
        locator = Nominatim(user_agent="myGeocoder")
        location = locator.geocode(jobs[i])
        lat.append(location.latitude)
        long.append(location.longitude)
    json_data = []
    lurl = ""
    for i in range(len(jobs)):
        lurl += f'{long[i]},{lat[i]};'
    url = 'http://127.0.0.1:5000/route/v1/driving/'
    url += lurl[:-1]
    r = requests.get(url)
    json_data.append(r.json())
    if ("routes" not in json_data[0]):
        return -1
    return json_data[0]['routes'][0]['duration']


#Calculate total time of schedules generated by routing function
def calculate_timex(arr,data):
    job_time = 0
    for i in arr:
        job_time += i.time
    addresses = [data.location]
    for j in arr:
        addresses.append(j.location)
    addresses.append(data.location)
    schedule = []
    print("\nEngineer {}: ".format(data.eid))
    for i in arr:
        schedule.append(i.jid)
    print(schedule)
    travel_time = osrm_route(addresses)
    if (travel_time == -1):
        print("Route not Possible")  
        return -1
    travel_time = travel_time/60
    print("\nTotal Time Required : {}".format(job_time + travel_time))
    return job_time + travel_time

#Function that calculates statistical data
def cal_stats(data):
    time = 0
    minimum = sys.float_info.max
    maximum = sys.float_info.min
    for i in range(len(data["engg"])):
        arr = []
        for k in range(len(data["engg"][i].schedule)):
            arr.append(get_job(data["engg"][i].schedule[k],data["jobs"]))
        temp = calculate_timex(arr,data["engg"][i])
        minimum = min(minimum,temp)
        maximum = max(maximum,temp)
        time+=temp
    print("\nMaximum Time: {} minutes \nMinimum Time: {} minutes \nRange: {} minutes\nTotal Time: {} minutes \n".format(maximum,minimum,maximum-minimum,time))

#Data Processing Function
def data_pro(data,job):
    print("Initiating Task Move with TSP: \n")
    qualified = get_qualified(data,job)
    if not qualified:
        print("No Engineer is qualified")
    x = task_move_op_tsp(data,job,qualified)
    if(x == -1):
        print("Job J not assigned")

icalculate_time(data)
print("\n")
print("Initial Schedule: ")
for i in data["engg"]:
    i.print()
cal_stats(data)
print("\n")
result = data_pro(data,Job)
print("Final Schedule: ")
for i in data["engg"]:
    i.print()
cal_stats(data)

et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')
eet = time.process_time()
res = eet - sst
print('CPU Execution time:', res, 'seconds')


Addresses of the Jobs: 
['B21 9RJ', 'B79 7PB', 'CV9 1LQ', 'B46 1AN', 'B21 9RJ']

Addresses of the Jobs: 
['B68 0LH', 'WV4 6ED', 'WV7 3BW', 'DY11 5SW', 'B68 0LH']

Addresses of the Jobs: 
['B11 1LZ', 'B96 6BD', 'B98 9EY', 'B90 8AT', 'B11 1LZ']

Addresses of the Jobs: 
['B25 8RN', 'NN1 5BD', 'B25 8RN']


Initial Schedule: 
------------------------------------
E.ID = 1
Schedule = ['3', '10', '5']
Total Time = 360.7 minutes
------------------------------------
E.ID = 2
Schedule = ['2', '9', '1']
Total Time = 447.545 minutes
------------------------------------
E.ID = 3
Schedule = ['8', '6', '4']
Total Time = 442.005 minutes
------------------------------------
E.ID = 4
Schedule = ['7']
Total Time = 375.03666666666663 minutes

Engineer 1: 
['3', '10', '5']

Addresses of the Jobs: 
['B21 9RJ', 'B79 7PB', 'CV9 1LQ', 'B46 1AN', 'B21 9RJ']

Total Time Required : 360.7

Engineer 2: 
['2', '9', '1']

Addresses of the Jobs: 
['B68 0LH', 'WV4 6ED', 'WV7 3BW', 'DY11 5SW', 'B68 0LH']

Total Time Req